In [2]:
import numpy as np
import pandas as pd 
import seaborn as sns
import datetime as dt
from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import random
from pandas_profiling import ProfileReport
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error as MSE

In [3]:
energy = pd.read_csv(r"C:\Users\Admin\OneDrive\Desktop\KAG_energydata_complete.csv")
energy.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [4]:
#converting date into datetime
energy['date'] = energy['date'].astype('datetime64[ns]')
energy.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [5]:
#making the date, time, hour and month into a diff column and thus converting them into date time format
energy['Date'] = pd.to_datetime(energy['date']).dt.date
energy['Time'] = pd.to_datetime(energy['date']).dt.time
energy['hour'] = energy['date'].dt.hour
energy['month'] = energy['date'].dt.month

In [6]:
energy.drop(['Date', 'Time'], axis = 1 , inplace = True)

In [7]:
energy.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2,hour,month
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433,17,1
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195,17,1
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668,17,1
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389,17,1
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097,17,1


In [8]:
energy1 = energy.iloc[ : , 1:31]
energy1

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2,hour,month
0,60,30,19.890000,47.596667,19.200000,44.790000,19.790000,44.730000,19.000000,45.566667,...,6.600000,733.5,92.000000,7.000000,63.000000,5.300000,13.275433,13.275433,17,1
1,60,30,19.890000,46.693333,19.200000,44.722500,19.790000,44.790000,19.000000,45.992500,...,6.483333,733.6,92.000000,6.666667,59.166667,5.200000,18.606195,18.606195,17,1
2,50,30,19.890000,46.300000,19.200000,44.626667,19.790000,44.933333,18.926667,45.890000,...,6.366667,733.7,92.000000,6.333333,55.333333,5.100000,28.642668,28.642668,17,1
3,50,40,19.890000,46.066667,19.200000,44.590000,19.790000,45.000000,18.890000,45.723333,...,6.250000,733.8,92.000000,6.000000,51.500000,5.000000,45.410389,45.410389,17,1
4,60,40,19.890000,46.333333,19.200000,44.530000,19.790000,45.000000,18.890000,45.530000,...,6.133333,733.9,92.000000,5.666667,47.666667,4.900000,10.084097,10.084097,17,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19730,100,0,25.566667,46.560000,25.890000,42.025714,27.200000,41.163333,24.700000,45.590000,...,22.733333,755.2,55.666667,3.333333,23.666667,13.333333,43.096812,43.096812,17,5
19731,90,0,25.500000,46.500000,25.754000,42.080000,27.133333,41.223333,24.700000,45.590000,...,22.600000,755.2,56.000000,3.500000,24.500000,13.300000,49.282940,49.282940,17,5
19732,270,10,25.500000,46.596667,25.628571,42.768571,27.050000,41.690000,24.700000,45.730000,...,22.466667,755.2,56.333333,3.666667,25.333333,13.266667,29.199117,29.199117,17,5
19733,420,10,25.500000,46.990000,25.414000,43.036000,26.890000,41.290000,24.700000,45.790000,...,22.333333,755.2,56.666667,3.833333,26.166667,13.233333,6.322784,6.322784,17,5


In [9]:
#creating feature vector
feature = energy1.iloc[:, 1:30]
feature

,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,...,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2,hour,month
0,30,19.890000,47.596667,19.200000,44.790000,19.790000,44.730000,19.000000,45.566667,17.166667,...,6.600000,733.5,92.000000,7.000000,63.000000,5.300000,13.275433,13.275433,17,1
1,30,19.890000,46.693333,19.200000,44.722500,19.790000,44.790000,19.000000,45.992500,17.166667,...,6.483333,733.6,92.000000,6.666667,59.166667,5.200000,18.606195,18.606195,17,1
2,30,19.890000,46.300000,19.200000,44.626667,19.790000,44.933333,18.926667,45.890000,17.166667,...,6.366667,733.7,92.000000,6.333333,55.333333,5.100000,28.642668,28.642668,17,1
3,40,19.890000,46.066667,19.200000,44.590000,19.790000,45.000000,18.890000,45.723333,17.166667,...,6.250000,733.8,92.000000,6.000000,51.500000,5.000000,45.410389,45.410389,17,1
4,40,19.890000,46.333333,19.200000,44.530000,19.790000,45.000000,18.890000,45.530000,17.200000,...,6.133333,733.9,92.000000,5.666667,47.666667,4.900000,10.084097,10.084097,17,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19730,0,25.566667,46.560000,25.890000,42.025714,27.200000,41.163333,24.700000,45.590000,23.200000,...,22.733333,755.2,55.666667,3.333333,23.666667,13.333333,43.096812,43.096812,17,5
19731,0,25.500000,46.500000,25.754000,42.080000,27.133333,41.223333,24.700000,45.590000,23.230000,...,22.600000,755.2,56.000000,3.500000,24.500000,13.300000,49.282940,49.282940,17,5
19732,10,25.500000,46.596667,25.628571,42.768571,27.050000,41.690000,24.700000,45.730000,23.230000,...,22.466667,755.2,56.333333,3.666667,25.333333,13.266667,29.199117,29.199117,17,5
19733,10,25.500000,46.990000,25.414000,43.036000,26.890000,41.290000,24.700000,45.790000,23.200000,...,22.333333,755.2,56.666667,3.833333,26.166667,13.233333,6.322784,6.322784,17,5


In [10]:
#splitting into test and train
X = feature
y = energy['Appliances']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=40)

In [23]:
from sklearn.metrics import mean_squared_error as MSE
# fit the model
model = ExtraTreesRegressor()
model.fit(X_train, y_train)
# evaluate the model
pred = model.predict(X_test) 
# evaluate predictions
rmse = np.sqrt(MSE(y_test, pred))
print("RMSE : % f" %(rmse))
mae = mean_absolute_error(y_test,pred)
print("MAE : % f" %(mae))
print("R2 : % f" %(model.score(X_test,y_test)))

RMSE :  65.386865
MAE :  31.108249
R2 :  0.583392


In [20]:
sc = StandardScaler()
X_train[['lights','T1','RH_1','T2','RH_2','T3','RH_3','T4','RH_4','T5','RH_5','T6','RH_6','T7','RH_7','T8','RH_8','T9','RH_9','T_out','Press_mm_hg','RH_out','Windspeed','Visibility','Tdewpoint','rv1','rv2', 'hour', 'month']] = sc.fit_transform(X_train[['lights','T1','RH_1','T2','RH_2','T3','RH_3','T4','RH_4','T5','RH_5','T6','RH_6','T7','RH_7','T8','RH_8','T9','RH_9','T_out','Press_mm_hg','RH_out','Windspeed','Visibility','Tdewpoint','rv1','rv2', 'hour', 'month']])
X_train_z = X_train
X_test[['lights','T1','RH_1','T2','RH_2','T3','RH_3','T4','RH_4','T5','RH_5','T6','RH_6','T7','RH_7','T8','RH_8','T9','RH_9','T_out','Press_mm_hg','RH_out','Windspeed','Visibility','Tdewpoint','rv1','rv2', 'hour', 'month']] = sc.fit_transform(X_test[['lights','T1','RH_1','T2','RH_2','T3','RH_3','T4','RH_4','T5','RH_5','T6','RH_6','T7','RH_7','T8','RH_8','T9','RH_9','T_out','Press_mm_hg','RH_out','Windspeed','Visibility','Tdewpoint','rv1','rv2', 'hour', 'month']])
X_test_z = X_test

In [21]:
X_train_z.head()

,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,...,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2,hour,month
9544,2.039003,0.569827,-1.398755,-0.383584,-0.629413,-0.384018,-1.197521,0.996241,-1.234796,-0.229230,...,-0.978251,1.167725,0.529878,-1.112160,1.922273,-0.803979,1.250473,1.250473,1.667248,-0.077382
19366,-0.479715,1.254979,0.004687,0.298556,0.604569,1.561061,0.228904,1.344290,-0.009491,0.163277,...,-0.217827,0.226014,0.931169,-1.248270,-1.104045,0.365687,1.468790,1.468790,-1.081917,1.416856
10816,2.039003,0.320681,1.388043,0.054501,1.101918,0.366012,-0.212353,-0.229283,1.045908,0.145189,...,1.120770,-0.733721,-0.896934,0.929492,0.140422,0.803317,0.048731,0.048731,1.088477,-0.077382
15457,-0.479715,-0.246127,-1.125096,-0.961129,-0.629413,-0.489022,-1.367589,-0.159019,-0.852893,-0.162305,...,-1.339609,-0.046587,0.975757,-0.839939,-1.542436,-1.006881,-1.459682,-1.459682,-1.515996,0.669737
2956,0.779644,-0.246127,1.789743,-0.024324,1.680118,-0.300681,2.300361,-1.013618,2.332439,-0.061616,...,0.630579,0.419763,0.864287,1.065602,1.752573,1.400086,-1.121664,-1.121664,-0.937224,-0.824501


In [24]:
# fit the model
model = ExtraTreesRegressor()
model.fit(X_train_z, y_train)
# evaluate the model
pred = model.predict(X_test_z)
# evaluate predictions
rmse = np.sqrt(MSE(y_test, pred))
print("RMSE : % f" %(rmse))
mae = mean_absolute_error(y_test,pred)
print("MAE : % f" %(mae))
print("R2 : % f" %(model.score(X_test_z,y_test)))

RMSE :  65.592228
MAE :  31.524564
R2 :  0.580771


In [32]:
from sklearn.model_selection import RepeatedKFold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
# define the evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define the pipeline to evaluate
model = ExtraTreesRegressor()
fs = SelectKBest(score_func=mutual_info_regression)
pipeline = Pipeline(steps=[('sel',fs), ('lr', model)])
# define the grid
grid = dict()
grid['sel__k'] = [i for i in range(X.shape[1]-20, X.shape[1]+1)]
# define the grid search
search = GridSearchCV(pipeline, grid, scoring='r2', n_jobs=-1, cv=cv)
# perform the search
results = search.fit(X, y)
# summarize best
print('Best RMSE: %.3f' % results.best_score_)
print('Best Config: %s' % results.best_params_)


Best RMSE: 0.659
Best Config: {'sel__k': 21}


In [33]:
pca = PCA(0.95)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [34]:
# fit the model
model = ExtraTreesRegressor()
model.fit(X_train_pca, y_train)
# evaluate the model
pred = model.predict(X_test_pca)
# evaluate predictions
rmse = np.sqrt(MSE(y_test, pred))
print("RMSE : % f" %(rmse))
mae = mean_absolute_error(y_test,pred)
print("MAE : % f" %(mae))
print("R2 : % f" %(model.score(X_test_pca,y_test)))

RMSE :  77.675638
MAE :  40.078192
R2 :  0.412083


In [15]:
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV, RepeatedKFold
import numpy as np


feature_names = ('lights','T1','RH_1','T2','RH_2','T3','RH_3','T4','RH_4',
                 'T5','RH_5','T6','RH_6','T7','RH_7','T8','RH_8','T9','RH_9','T_out',
                 'Press_mm_hg','RH_out','Windspeed','Visibility','Tdewpoint','rv1','rv2', 'hour', 'month')

# Define the feature selection and multiple regression pipeline
pipe = Pipeline([('select', SelectKBest(score_func=mutual_info_regression)),
                 ('regress', ExtraTreesRegressor())])

# Define the parameter grid for the number of features to select
param_grid = {'select__k': range(1, X.shape[1] + 1)}

# Define the cross-validation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

# Define the evaluation metrics
metrics = ['neg_mean_squared_error', 'neg_mean_absolute_error', 'r2']

# Define the GridSearchCV object
grid = GridSearchCV(pipe, param_grid=param_grid, cv=cv, scoring=metrics, refit='neg_mean_squared_error', return_train_score=True)

# Fit the grid search to the data
grid.fit(X_train, y_train)

# Get the best number of features
best_k = grid.best_params_['select__k']

# Get the selected feature indices and names
selected_indices = grid.best_estimator_.named_steps['select'].get_support(indices=True)
selected_features = [feature_names[i] for i in selected_indices]

# Print the best number of features and the selected feature names
print("Best number of features:", best_k)
print("Selected features:", selected_features)

# Get the best estimator
best_estimator = grid.best_estimator_

# Get the predictions on the entire dataset
y_pred = best_estimator.predict(X_test)

# Calculate the evaluation metrics on the entire dataset
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print("RMSE:", rmse)
print("MAE:", mae)
print("R2:", r2)

Best number of features: 21
Selected features: ['T1', 'RH_1', 'T2', 'T3', 'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'hour']
RMSE: 61.09535388153515
MAE: 27.356546412646942
R2: 0.6362837706273023


In [16]:
y_pred = best_estimator.predict(X_train)

# Calculate the evaluation metrics on the entire dataset
rmse = np.sqrt(mean_squared_error(y_train, y_pred))
mae = mean_absolute_error(y_train, y_pred)
r2 = r2_score(y_train, y_pred)

# Print the evaluation metrics
print("RMSE:", rmse)
print("MAE:", mae)
print("R2:", r2)

RMSE: 0.010696106031310906
MAE: 0.0004729410174988071
R2: 0.9999999892003113


In [19]:
from sklearn.datasets import make_regression
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, RepeatedKFold,train_test_split
from math import sqrt

# Initialize the model
model = ExtraTreesRegressor()

# Initialize RFE
rfe = RFE(model, step=1)

# Define the parameter grid
param_grid = {'n_estimators': [50, 100, 200],
              'max_depth': [None, 5, 10]}

# Initialize the repeated K-fold cross-validator
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=cv, n_jobs=-1, verbose=1)

# Fit GridSearchCV on RFE-transformed data
grid_search.fit(rfe.fit_transform(X_train, y_train), y_train)

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters:", best_params)

# Get the selected features
selected_features = rfe.get_support(indices=True)

# Get the feature names
feature_names = ('lights','T1','RH_1','T2','RH_2','T3','RH_3','T4','RH_4',
                 'T5','RH_5','T6','RH_6','T7','RH_7','T8','RH_8','T9','RH_9','T_out',
                 'Press_mm_hg','RH_out','Windspeed','Visibility','Tdewpoint','rv1','rv2', 'hour', 'month')
selected_feature_names = [feature_names[i] for i in selected_features]

# Print the selected feature names
print("Selected features:", selected_feature_names)

# Get the number of selected features
n_features = rfe.n_features_
print("Number of selected features:", n_features)

# Get the predicted values
y_pred = grid_search.predict(rfe.transform(X_test))

# Calculate evaluation metrics
rmse = sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print evaluation metrics
print("RMSE:", rmse)
print("MAE:", mae)
print("R2:", r2)

Fitting 30 folds for each of 9 candidates, totalling 270 fits
Best parameters: {'max_depth': None, 'n_estimators': 200}
Selected features: ['T1', 'RH_1', 'RH_2', 'T3', 'RH_3', 'T4', 'T6', 'T8', 'RH_8', 'RH_9', 'RH_out', 'Windspeed', 'Tdewpoint', 'hour']
Number of selected features: 14
RMSE: 61.18320646421537
MAE: 27.370561410620187
R2: 0.6352370008839353


In [20]:
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV, RepeatedKFold
import numpy as np


feature_names = ('lights','T1','RH_1','T2','RH_2','T3','RH_3','T4','RH_4',
                 'T5','RH_5','T6','RH_6','T7','RH_7','T8','RH_8','T9','RH_9','T_out',
                 'Press_mm_hg','RH_out','Windspeed','Visibility','Tdewpoint','rv1','rv2', 'hour', 'month')

# Define the feature selection and multiple regression pipeline
pipe = Pipeline([('select', SelectKBest(score_func=f_regression)),
                 ('regress', ExtraTreesRegressor())])

# Define the parameter grid for the number of features to select
param_grid = {'select__k': range(1, X.shape[1] + 1)}

# Define the cross-validation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

# Define the evaluation metrics
metrics = ['neg_mean_squared_error', 'neg_mean_absolute_error', 'r2']

# Define the GridSearchCV object
grid = GridSearchCV(pipe, param_grid=param_grid, cv=cv, scoring=metrics, refit='neg_mean_squared_error', return_train_score=True)

# Fit the grid search to the data
grid.fit(X_train, y_train)

# Get the best number of features
best_k = grid.best_params_['select__k']

# Get the selected feature indices and names
selected_indices = grid.best_estimator_.named_steps['select'].get_support(indices=True)
selected_features = [feature_names[i] for i in selected_indices]

# Print the best number of features and the selected feature names
print("Best number of features:", best_k)
print("Selected features:", selected_features)

# Get the best estimator
best_estimator = grid.best_estimator_

# Get the predictions on the entire dataset
y_pred = best_estimator.predict(X_test)

# Calculate the evaluation metrics on the entire dataset
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print("RMSE:", rmse)
print("MAE:", mae)
print("R2:", r2)

Best number of features: 19
Selected features: ['lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4', 'T6', 'RH_6', 'RH_7', 'T8', 'RH_8', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'hour']
RMSE: 64.85889176591405
MAE: 29.63421159302797
R2: 0.5900929780780848
